In [ ]:
import gradio as gr
import openai

openai.api_key = ""

def predict(message, history):
    # OpenAI Chatgpt API에 전달할 메세지 형식
    history_openai_format = []

    # 기존의 대화 목록을 OpenAI Chatgpt가 이해할 수 있도록 변환
    for human, assistant in history:
        history_openai_format.append({"role":"user", "content": human}) # 사용자의 메세지를 추가합니다.
        history_openai_format.append({"role":"assistant", "content": assistant}) # chatgpt의 메세지를 추가합니다.

    # 가장 최근에 입력된 사용자의 메시지를 추가합니다.
    history_openai_format.append({"role":"user", "content": message})

    # OPENAI ChatGPT API 호출
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", # 어떤 모델(인공지능)을 쓸지 결정
        messages=[{"role":"system", "content": "당신은 웨이트 트레이닝 전문가입니다. 웨이트 트레이닝 이외의 답변은 하지 마시오, 헬스 관련 내용은 친절하게 이모티콘을 사용해서 자세히 답변해줘."}, *history_openai_format], # 보낼 메세지(대화 기록 전송)
        temperature=0.8, # 모델의 응답을 얼마나 다양하게 할건지 결정(0~1 사이의 값) -> 0이면 일관된 답변, 1이면 창의적인 답변
        stream=True # 응답을 스트리밍 형태로 받을 것인지 여부
    )
    partial_message = ""
    for chunk in response:
        if len(chunk['choices'][0]['delta']) != 0:
            partial_message += chunk['choices'][0]['delta']['content']
            yield partial_message

# 챗봇 UI 설정
gr.ChatInterface(
    fn=predict,
    textbox=gr.Textbox(placeholder="말 걸어 주실래요?", container=False, scale=7),
    title="헬짱봇",
    description="헬스/운동 내용만 답변합니다.",
    theme="soft",
    retry_btn="다시보내기 ↩",
    undo_btn="이전챗 삭제 ❌",
    clear_btn="전챗 삭제 💫",
    examples=[["안녕"], ["인공지능 공부 어떻게 해?"], ["다이어트 꿀팁좀"]]
).queue().launch(share=True, debug=True)